In [2]:
import os
import json
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
# load .env
load_dotenv(find_dotenv())

In [3]:
# Turn off SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

In [20]:
directory = "data"

# url kbli
kbli_url = os.getenv('BASE_URL_KBLI_BPS')

In [22]:
# read json files
def read_json_files():
  # Daftar semua file dalam direktori
  files = os.listdir(f"{directory}/raw")
  # Filter hanya file JSON
  json_files = [file for file in files if file.endswith('.json')]
  # Loop untuk membaca setiap file JSON
  data_list = []
  for file_name in json_files:
      file_path = os.path.join(f"{directory}/raw", file_name)
      
      # Buka dan baca file JSON
      with open(file_path, 'r') as json_file:
          data = json.load(json_file)
          # mengambil data
          json_data = data['data'][1]
          for jd in json_data:
            data_list.append(jd['_source'])  # Simpan data dari setiap file JSON ke dalam list

  return data_list

def extract_json_list(json_list):
    list_data = []
    for item in tqdm(json_list):
        data = {}
        data['id'] = item.get("id")
        data['jenis'] = item.get("jenis")
        data['sumber'] = item.get("source")
        data['kode_kbli'] = item.get("id").split("_")[2]
        data['judul'] = item.get("judul").split("] ")[1]
        data['last_update'] = item.get("last_update")
        data['url_kbli'] = item.get("url")
        data['tags'] = item.get("tags")

        list_sebelumnya = item.get("sebelumnya")
        for sebelumnya in list_sebelumnya:
            if (len(sebelumnya['kode']) == 4):
                data['subgolongan_kode'] = sebelumnya["kode"]
                data['subgolongan_judul'] = sebelumnya["judul"]
                data['subgolongan_url'] = f"{kbli_url}/{sebelumnya['kode']}"
            if (len(sebelumnya['kode']) == 3):
                data['golongan_kode'] = sebelumnya["kode"]
                data['golongan_judul'] = sebelumnya["judul"]
                data['golongan_url'] = f"{kbli_url}/{sebelumnya['kode']}"
            if (len(sebelumnya['kode']) == 2):
                data['golongan_pokok_kode'] = sebelumnya["kode"]
                data['golongan_pokok_judul'] = sebelumnya["judul"]
                data['golongan_pokok_url'] = f"{kbli_url}/{sebelumnya['kode']}"
            if (len(sebelumnya['kode']) == 1):
                data['kategori_kode'] = sebelumnya["kode"]
                data['kategori_judul'] = sebelumnya["judul"]
                data['kategori_url'] = f"{kbli_url}/{sebelumnya['kode']}"

        list_data.append(data)
    
    return pd.DataFrame(list_data)

In [23]:
list_data = read_json_files()
final_df = extract_json_list(list_data)
final_df.head()

100%|██████████| 1789/1789 [00:00<00:00, 79165.36it/s]


,id,jenis,sumber,kode_kbli,judul,last_update,url_kbli,tags,subgolongan_kode,subgolongan_judul,subgolongan_url,golongan_kode,golongan_judul,golongan_url,golongan_pokok_kode,golongan_pokok_judul,golongan_pokok_url,kategori_kode,kategori_judul,kategori_url
0,kbli_2020_01111,kbli2020,Metadata Management System (MMS),01111,Pertanian Jagung,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"[pertanian, agriculture]",0111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,011,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,01,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...
1,kbli_2020_01112,kbli2020,Metadata Management System (MMS),01112,Pertanian Gandum,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"[pertanian, agriculture]",0111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,011,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,01,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...
2,kbli_2020_01113,kbli2020,Metadata Management System (MMS),01113,Pertanian Kedelai,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"[pertanian, agriculture]",0111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,011,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,01,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...
3,kbli_2020_01114,kbli2020,Metadata Management System (MMS),01114,Pertanian Kacang Tanah,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"[pertanian, agriculture]",0111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,011,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,01,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...
4,kbli_2020_01115,kbli2020,Metadata Management System (MMS),01115,Pertanian Kacang Hijau,2023-12-21,https://klasifikasi.web.bps.go.id/app/view/kbl...,"[pertanian, agriculture]",0111,"Pertanian serealia (bukan padi), aneka kacang ...",https://klasifikasi.web.bps.go.id/app/view/kbl...,011,Pertanian Tanaman Semusim,https://klasifikasi.web.bps.go.id/app/view/kbl...,01,"Pertanian Tanaman, Peternakan, Perburuan dan K...",https://klasifikasi.web.bps.go.id/app/view/kbl...,A,"Pertanian, Kehutanan dan Perikanan",https://klasifikasi.web.bps.go.id/app/view/kbl...


In [24]:
 # Save to csv
kbli_file = os.path.join(f'{directory}/processed', 'kbli2020.csv')
final_df.to_csv(kbli_file, index=False)
print("Data saved to kbli2020.csv")

Data saved to kbli2020.csv


In [ ]:
# pemisahan KBLI menjadi beberapa entitas
def get_unique_kategori_kbli(df):
  selected_df = df.drop_duplicates(subset='kategori_kode', keep='first')
  selected_df = selected_df[['kategori_kode', 'kategori_judul', 'kategori_url']].rename(
    columns={
      'kategori_kode': 'kode',
      'kategori_judul': 'nama',
      'kategori_url': 'url'
    }
  )
  return selected_df

def get_unique_golongan_pokok_kbli(df):
  selected_df = df.drop_duplicates(subset='golongan_pokok_kode', keep='first')
  selected_df = selected_df[['golongan_pokok_kode', 'golongan_pokok_judul', 'golongan_pokok_url']].rename(
    columns={
      'golongan_pokok_kode': 'kode',
      'golongan_pokok_judul': 'nama',
      'golongan_pokok_url': 'url'
    }
  )
  return selected_df

def get_unique_golongan_kbli(df):
  selected_df = df.drop_duplicates(subset='golongan_kode', keep='first')
  selected_df = selected_df[['golongan_kode', 'golongan_judul', 'golongan_url']].rename(
    columns={
      'golongan_kode': 'kode',
      'golongan_judul': 'nama',
      'golongan_url': 'url'
    }
  )
  return selected_df

def get_unique_subgolongan_kbli(df):
  selected_df = df.drop_duplicates(subset='subgolongan_kode', keep='first')
  selected_df = selected_df[['subgolongan_kode', 'subgolongan_judul', 'subgolongan_url']].rename(
    columns={
      'subgolongan_kode': 'kode',
      'subgolongan_judul': 'nama',
      'subgolongan_url': 'url'
    }
  )
  return selected_df

def get_unique_kelompok_kbli(df):
  selected_df = df.drop_duplicates(subset='kode_kbli', keep='first')
  selected_df = selected_df[['kode_kbli', 'judul', 'url_kbli']].rename(
    columns={
      'kode_kbli': 'kode',
      'judul': 'nama',
      'url_kbli': 'url'
    }
  )
  return selected_df

def get_relasi_kelompok_kategori_kbli(df):
  selected_df = df.drop_duplicates(subset=['kode_kbli', 'kategori_kode'])
  selected_df = selected_df[['kode_kbli', 'kategori_kode']].rename(
    columns={
      'kode_kbli': 'kelompok_kbli_kode',
      'kategori_kode': 'kategori_kbli_kode'
    }
  )
  return selected_df

def get_relasi_kelompok_golongan_pokok_kbli(df):
  selected_df = df.drop_duplicates(subset=['kode_kbli', 'golongan_pokok_kode'])
  selected_df = selected_df[['kode_kbli', 'golongan_pokok_kode']].rename(
    columns={
      'kode_kbli': 'kelompok_kbli_kode',
      'golongan_pokok_kode': 'golongan_pokok_kbli_kode'
    }
  )
  return selected_df

def get_relasi_kelompok_golongan_kbli(df):
  selected_df = df.drop_duplicates(subset=['kode_kbli', 'golongan_kode'])
  selected_df = selected_df[['kode_kbli', 'golongan_kode']].rename(
    columns={
      'kode_kbli': 'kelompok_kbli_kode',
      'golongan_kode': 'golongan_kbli_kode'
    }
  )
  return selected_df

def get_relasi_kelompok_subgolongan_kbli(df):
  selected_df = df.drop_duplicates(subset=['kode_kbli', 'subgolongan_kode'])
  selected_df = selected_df[['kode_kbli', 'subgolongan_kode']].rename(
    columns={
      'kode_kbli': 'kelompok_kbli_kode',
      'subgolongan_kode': 'subgolongan_kbli_kode'
    }
  )
  return selected_df

def get_relasi_perusahaan_kelompok_kbli():
  df = pd.read_csv('../establishments/data/processed/siinas_data.csv')
  selected_df = df.drop_duplicates(subset=['id', 'kbli'])
  selected_df = selected_df[['id', 'kbli']].rename(
    columns={
      'id': 'id_perusahaan',
      'kbli': 'kelompok_kbli_kode',
    }
  )
  return selected_df

def save_to_csv(df, filename):
  df.to_csv(f"../../datasets/kbli/{filename}", index=False)
  print(f"{filename} has been saved to datasets")

In [ ]:
save_to_csv(get_unique_kategori_kbli(final_df), "kategori_kbli.csv")
save_to_csv(get_unique_golongan_pokok_kbli(final_df), "golongan_pokok_kbli.csv")
save_to_csv(get_unique_golongan_kbli(final_df), "golongan_kbli.csv")
save_to_csv(get_unique_subgolongan_kbli(final_df), "subgolongan_kbli.csv")
save_to_csv(get_unique_kelompok_kbli(final_df), "kelompok_kbli.csv")
save_to_csv(get_relasi_kelompok_kategori_kbli(final_df), "relasi_kelompok_kategori_kbli.csv")
save_to_csv(get_relasi_kelompok_golongan_pokok_kbli(final_df), "relasi_kelompok_golongan_pokok_kbli.csv")
save_to_csv(get_relasi_kelompok_golongan_kbli(final_df), "relasi_kelompok_golongan_kbli.csv")
save_to_csv(get_relasi_kelompok_subgolongan_kbli(final_df), "relasi_kelompok_subgolongan_kbli.csv")
save_to_csv(get_relasi_perusahaan_kelompok_kbli(), "relasi_perusahaan_kelompok_kbli.csv")